In [ ]:
!pip uninstall sklearn -y
!pip install --upgrade sklearn
!pip install scikit-learn==0.23.2 --user
# 설치 후 런타임 다시시작

In [ ]:
import sklearn
!pip install pycaret
from pycaret.utils import enable_colab
enable_colab()
!pip install markupsafe==2.0.1
# 설치 후 런타임 다시시작

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 KB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11953 sha256=7a8d5605042e66a903d4c076c6fb24bac596865f6fdaaa0ceaa23cb53e16c340
  Stored in directory: /root/.cache/pip/wheels/a8/bb/8f/6b9328d23c2dcedbfeb8498b9f650d55d463089e3b8fc0bfb2
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [3]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 3.3 MB/s eta 0:00:00


In [35]:
import sys, os, os.path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import os
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

from category_encoders.cat_boost import CatBoostEncoder

from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

train = pd.read_csv('./drive/MyDrive/input/유전체 정보 품종 분류 AI 경진대회/train.csv')
test = pd.read_csv('./drive/MyDrive/input/유전체 정보 품종 분류 AI 경진대회/test.csv')
train

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


### Feature Engineering

In [36]:
# 1. SNP_first/last 변수 생성
def first_last(data):
    # SNP_01 ~ SNP_09
    for i in range(1, 10):
        data['SNP_0{0}_first'.format(i)] = data['SNP_0{0}'.format(i)].str.get(0)
        data['SNP_0{0}_last'.format(i)] = data['SNP_0{0}'.format(i)].str.get(2)
    # SNP_10 ~ SNP_15
    for i in range(10, 16):
        data['SNP_{0}_first'.format(i)] = data['SNP_{0}'.format(i)].str.get(0)
        data['SNP_{0}_last'.format(i)] = data['SNP_{0}'.format(i)].str.get(2)    
    return data   

In [37]:
train = first_last(train)
test = first_last(test)
train

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_11_first,SNP_11_last,SNP_12_first,SNP_12_last,SNP_13_first,SNP_13_last,SNP_14_first,SNP_14_last,SNP_15_first,SNP_15_last
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A,G,A,A,A,A,A,A,A,A
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A,A,G,A,G,G,A,A,A,A
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A,A,A,A,A,A,A,A,A,A
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G,G,G,G,G,G,A,A,G,G
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A,A,A,A,A,G,A,A,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A,G,G,A,A,A,A,A,A,A
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,A,G,A,A,A,G,A,A,G,A
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G,G,G,G,G,G,C,A,G,G
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,A,G,G,A,G,G,C,A,G,G


In [38]:
# 2. 정답label 분리
def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

In [39]:
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)
train_x

,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,...,SNP_11_first,SNP_11_last,SNP_12_first,SNP_12_last,SNP_13_first,SNP_13_last,SNP_14_first,SNP_14_last,SNP_15_first,SNP_15_last
0,0,0,0,2,G G,A G,A A,G A,C A,A A,...,A,G,A,A,A,A,A,A,A,A
1,0,0,0,2,A G,A G,C A,A A,A A,A G,...,A,A,G,A,G,G,A,A,A,A
2,0,0,0,2,G G,G G,A A,G A,C C,G G,...,A,A,A,A,A,A,A,A,A,A
3,0,0,0,1,A A,G G,A A,G A,A A,G G,...,G,G,G,G,G,G,A,A,G,G
4,0,0,0,2,G G,G G,C C,A A,C C,A A,...,A,A,A,A,A,G,A,A,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0,0,0,2,A G,A G,A A,G A,C C,A G,...,A,G,G,A,A,A,A,A,A,A
258,0,0,0,2,G G,A A,C A,A A,A A,A G,...,A,G,A,A,A,G,A,A,G,A
259,0,0,0,1,A G,G G,A A,G A,A A,A G,...,G,G,G,G,G,G,C,A,G,G
260,0,0,0,1,A A,G G,A A,G A,A A,G G,...,A,G,G,A,G,G,C,A,G,G


In [40]:
# 3. LabelEncoder
class_le = preprocessing.LabelEncoder()
train_y = class_le.fit_transform(train_y)
train_y

array([1, 2, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 0, 0, 0, 0, 2, 1, 1,
       0, 1, 2, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 1, 1, 1, 0,
       2, 1, 1, 0, 2, 0, 0, 2, 0, 2, 1, 1, 0, 0, 2, 2, 1, 0, 2, 2, 0, 2,
       2, 0, 1, 2, 2, 2, 0, 1, 1, 1, 2, 1, 2, 0, 1, 1, 2, 0, 2, 0, 2, 2,
       2, 1, 1, 1, 2, 2, 1, 1, 2, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 0,
       1, 2, 0, 1, 2, 0, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 0, 1, 0, 2, 2, 2,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 0, 0, 2, 2, 0, 0, 2, 2, 2,
       2, 0, 2, 0, 0, 2, 0, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 0, 1, 1, 2,
       1, 2, 0, 1, 0, 1, 0, 2, 2, 1, 2, 1, 2, 1, 0, 0, 1, 2, 0, 0, 2, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 2, 1, 2, 1, 2, 1, 0, 2, 0, 0, 2,
       2, 1, 1, 1, 0, 1, 0, 1, 0, 2, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1])

In [41]:
# 3. LabelEncoder
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]
snp = []
for col in snp_col:
    snp += list(train_x[col].values)

snp_le.fit(snp)
    
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

train_x['label']=train_y

train_x=train_x.iloc[:,3:]
test_x=test_x.iloc[:,3:]
train_x.shape

(262, 47)

In [42]:
train_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,...,SNP_11_last,SNP_12_first,SNP_12_last,SNP_13_first,SNP_13_last,SNP_14_first,SNP_14_last,SNP_15_first,SNP_15_last,label
0,2,5,1,0,4,2,0,0,5,0,...,G,A,A,A,A,A,A,A,A,1
1,2,1,1,2,0,0,1,0,4,0,...,A,G,A,G,G,A,A,A,A,2
2,2,5,5,0,4,3,5,0,4,4,...,A,A,A,A,A,A,A,A,A,1
3,1,0,5,0,4,0,5,5,0,5,...,G,G,G,G,G,A,A,G,G,0
4,2,5,5,3,0,3,0,0,0,0,...,A,A,A,A,G,A,A,G,A,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,1,1,0,4,3,1,0,4,0,...,G,G,A,A,A,A,A,A,A,1
258,2,5,0,2,0,0,1,4,4,0,...,G,A,A,A,G,A,A,G,A,2
259,1,1,5,0,4,0,1,5,4,4,...,G,G,G,G,G,C,A,G,G,0
260,1,0,5,0,4,0,5,5,0,4,...,G,G,A,G,G,C,A,G,G,0


In [53]:
# 4. CatBoostEncoder
cat_train_df = train_x.copy()
cat_test_df = train_y.copy()

ce = CatBoostEncoder()

columns_list=list(cat_train_df.columns)
columns_list.remove('label')

train_x[columns_list] = ce.fit_transform(train_x[columns_list], train_x['label'])
test_x[columns_list] = ce.transform(test_x[columns_list])
train_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,...,SNP_11_last,SNP_12_first,SNP_12_last,SNP_13_first,SNP_13_last,SNP_14_first,SNP_14_last,SNP_15_first,SNP_15_last,label
0,2,5,1,0,4,2,0,0,5,0,...,1.038168,1.038168,1.038168,1.038168,1.038168,1.038168,1.038168,1.038168,1.038168,1
1,2,1,1,2,0,0,1,0,4,0,...,1.038168,1.038168,1.019084,1.038168,1.038168,1.019084,1.019084,1.019084,1.019084,2
2,2,5,5,0,4,3,5,0,4,4,...,1.519084,1.019084,1.346056,1.019084,1.019084,1.346056,1.346056,1.346056,1.346056,1
3,1,0,5,0,4,0,5,5,0,5,...,1.019084,1.519084,1.038168,1.519084,1.519084,1.259542,1.259542,1.038168,1.038168,0
4,2,5,5,3,0,3,0,0,0,0,...,1.346056,1.012723,1.259542,1.012723,1.012723,1.007634,1.007634,0.519084,1.259542,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,1,1,0,4,3,1,0,4,0,...,0.789990,0.645469,1.276884,1.301631,1.173811,1.262504,1.127822,1.262039,1.219698,1
258,2,5,0,2,0,0,1,4,4,0,...,0.791176,1.407690,1.275547,1.299579,1.009846,1.261077,1.127280,0.888409,1.218632,2
259,1,1,5,0,4,0,1,5,4,4,...,0.797967,0.648305,0.132796,0.708302,1.014607,0.513660,1.130963,0.895674,0.371077,0
260,1,0,5,0,4,0,5,5,0,4,...,0.793509,0.643160,1.279030,0.702089,1.009752,0.506989,1.126211,0.889858,0.364330,0


### train/test split

In [54]:
from sklearn.model_selection import train_test_split

X = train_x.drop(columns=['label'])
y = train_x['label']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=2021)

In [55]:
X_train.head()

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,...,SNP_11_first,SNP_11_last,SNP_12_first,SNP_12_last,SNP_13_first,SNP_13_last,SNP_14_first,SNP_14_last,SNP_15_first,SNP_15_last
26,2,5,5,3,4,3,0,0,4,0,...,1.239913,1.419847,0.503817,1.219051,1.265167,1.047189,1.183553,1.078391,1.233705,1.296363
3,1,0,5,0,4,0,5,5,0,5,...,1.038168,1.019084,1.519084,1.038168,1.519084,1.519084,1.259542,1.259542,1.038168,1.038168
41,2,5,1,2,0,3,1,0,5,0,...,1.201272,0.681527,0.502120,1.177593,1.186599,0.910248,1.086805,1.000954,1.151908,1.173730
48,2,5,1,2,0,2,0,4,4,0,...,1.207005,1.317798,1.270699,1.150954,1.157443,1.103817,1.100954,1.000830,1.175573,1.177593
35,2,0,0,0,0,2,0,0,4,0,...,1.186599,1.335878,1.287532,1.173730,1.168257,1.004771,1.067939,1.001123,0.738851,1.168257


### model

In [46]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.neural_network import MLPClassifier

In [56]:
# Average CV score on the training set was: 0.9631683010073815
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeClassifier(criterion="gini", max_depth=2, min_samples_leaf=12, min_samples_split=9)),
    StackingEstimator(estimator=MultinomialNB(alpha=1.0, fit_prior=True)),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="squared_hinge", penalty="l2", tol=0.0001)),
    GradientBoostingClassifier(learning_rate=0.1, max_depth=5, max_features=0.25, min_samples_leaf=18, min_samples_split=14, n_estimators=100, subsample=0.5)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

In [57]:
target=train_x['label']
new_train_df=train_x.iloc[:,:-1]

### predict

In [59]:
exported_pipeline.fit(new_train_df,target)
results = exported_pipeline.predict(test_x)

In [60]:
submit = pd.read_csv('./drive/MyDrive/input/유전체 정보 품종 분류 AI 경진대회/sample_submission.csv')
submit['class'] = class_le.inverse_transform(results)
submit['class']=results

submit['class']=submit['class'].apply(lambda x: 'A' if x == 0 else ( 'B' if x == 1 else 'C' ))
submit.to_csv('tpot3.csv', index=False)
submit